In [1]:
#import sklearn
#sklearn.__version__

'1.0.2'

In [ ]:
#pip uninstall scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.2


In [2]:
!pip install scikit-learn
!pip install fuzzywuzzy
!pip install Levenshtein
import nltk
nltk.download('stopwords')
#!pip install pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import Levenshtein
from fuzzywuzzy import process, fuzz

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset = 'drive/MyDrive/Bangkit 2022/Capstone Project/dice_com-job_us_sample.csv'

In [6]:
data = pd.read_csv(dataset)
df = data.copy()

In [7]:
df_skills = df.copy()
df_skills = df_skills[df_skills['skills'].notnull()]

In [8]:
def preprocess_skills(text):
  text = text.lower()

  tokenizer = RegexpTokenizer(r'\w+')
  text = tokenizer.tokenize(text)
  stop_words = stopwords.words('english')
  filtered_words = [word for word in text if word not in stop_words]

  return ' '.join(word for word in filtered_words)

In [9]:
df_skills['skills'] = df_skills['skills'].apply(lambda x: preprocess_skills(x))

In [10]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_skills['skills'])

In [12]:
import pickle
pickle.dump(tfidf_matrix, open('tfidf_vectorizer_skills.pickle', 'wb'))

In [13]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
df_skills = df_skills.reset_index()
skills = df_skills['skills']
indices = pd.Series(df_skills.index, index=df_skills['skills'])
df_skills.to_csv('jobs_recommends.csv')
indices.to_csv('skills_indices.csv')
skills.to_csv('skills.csv')

In [24]:
def get_most_similar_skills(skills):
  return indices[process.extractOne(skills, indices.index, scorer=fuzz.token_sort_ratio)[0]]

In [25]:
def get_recommendation(input_skills):
  idx = get_most_similar_skills(input_skills.lower())
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  job_indices = [i[0] for i in sim_scores]
  skills = df_skills['skills']
  return skills.iloc[job_indices]

In [26]:
def get_recommendation(input_skills, tfidf_matrix, df_skills, indices):
  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
  idx = indices[process.extractOne(input_skill.lower(), indices.index, scorer=fuzz.token_sort_ratio)[0]]

  if isinstance(idx, pd.Series):
    idx = idx[0]

  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  job_indices = [i[0] for i in sim_scores]
  return indices.iloc[job_indices], job_indices

In [31]:
#if __name__ == '__main__':
df_skills = pd.read_csv('skills.csv', usecols=['skills'])
main_df = pd.read_csv('jobs_recommends.csv', index_col='index')
indices = pd.read_csv('skills_indices.csv')
indices = pd.Series(indices.index, index=indices['skills'])

with open('tfidf_vectorizer_skills.pickle', 'rb') as f:
  tfidf_matrix = pickle.load(f)
input_skill = input('Enter the skills ')
recommendation, job_indices = get_recommendation(input_skill, tfidf_matrix, df_skills, indices)

Enter the skills python


In [32]:
main_df.iloc[job_indices, :].head()

,Unnamed: 0,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
index,,,,,,,,,,,,,
1338,1335,https://www.dice.com/jobs/detail/Software-Test...,Stowe Group,"C2H Independent, C2H W2, 6 Months to Perm",US Citizen and GC onlyProject Description: My...,Dice Id : 10291975A,"Manchester, NH",Software Test Engineer,1 week ago,Telecommuting not available|Travel not required,www.dice.com,c python,93cd95261c4f2cff338524547dd89641
1809,1806,https://www.dice.com/jobs/detail/Sr-Python-Dev...,Loganbritton,"Full Time, Contract Corp-To-Corp, Contract Ind...",• 5+ years as an Object Oriented Software Deve...,Dice Id : 10109326,"Boston, MA",Sr Python Developer,3 weeks ago,Telecommuting not available|Travel not required,www.dice.com,python,ba1347f82f63753afd2391ed51be81de
2854,2851,https://www.dice.com/jobs/detail/C%252B%252B-%...,COESYS Solutions Inc,"Contract Independent, Contract W2",C++ and python DeveloperDescriptionDesign and ...,Dice Id : 10220327,"Orangeburg, NY",C++ /Python Developer,3 weeks ago,Telecommuting not available|Travel not required,NaN,c python,144c5e14781a896e5a80f4263070e6fc
3401,3398,https://www.dice.com/jobs/detail/Python-Develo...,MphasiS Corporation USA,Full Time,We are seeking an hands-on technologist with ...,Dice Id : 501494924,"Brooklyn, NY",Python Developer,2 weeks ago,Telecommuting not available|Travel not required,NaN,python,fb653dca4c49045fa78962780717e10a
4554,4538,https://www.dice.com/jobs/detail/C%252B%252B-D...,United Software Group,"Full Time, Contract Corp-To-Corp, Contract Ind...","Hi,This is Siva from United Software Group Inc...",Dice Id : 10204540,"Redmond, WA",C++ Developer,2 hours ago,Telecommuting not available|Travel not required,NaN,c python,7bf931e53acbb984c4b88b3dbd6512c9
